In [42]:
import pandas as pd
import numpy as np

In [155]:
filename = "Data/RL_MDD_All.csv"
data_file = pd.read_csv(filename)


,Row Index,Subject,SubNum,Group,SubChoice,SubAction,RichFrac,Condition,RichFracChoice,Fractal1_Side,Fractal2_Side,Fractal_1_Rew,Fractal_2_Rew,TSCP,Reward
0,1,1210,1,MDD,2,2,1,Reward,0,1,2,1,0,1,0
1,2,1210,1,MDD,2,1,1,Reward,0,2,1,0,0,2,0
2,3,1210,1,MDD,1,1,1,Reward,1,1,2,1,0,3,1
3,4,1210,1,MDD,1,1,1,Reward,1,1,2,0,0,4,0
4,5,1210,1,MDD,2,2,1,Reward,0,1,2,0,0,5,0
5,6,1210,1,MDD,2,1,1,Reward,0,2,1,0,0,6,0
6,7,1210,1,MDD,1,2,1,Reward,1,2,1,0,0,7,0
7,8,1210,1,MDD,2,2,1,Reward,0,1,2,1,1,8,1
8,9,1210,1,MDD,2,2,1,Reward,0,1,2,1,0,9,0
9,10,1210,1,MDD,1,2,1,Reward,1,2,1,1,0,10,1


In [158]:
depressed = data_file.groupby(['Subject']).agg({'Group': lambda x: x.iloc[0]})
depressed

,Group
Subject,
1210,MDD
1211,MDD
1212,MDD
1213,MDD
1214,MDD
1215,MDD
1216,MDD
1217,MDD
1218,MDD


In [35]:
my_concat = lambda x: list(x.values)
agg_df = data_file.groupby(['Subject']).agg({
    'SubChoice':my_concat,
    'Reward' : my_concat,
    'RichFrac' : my_concat
}).reset_index()

In [38]:
agg_df

,Subject,Reward,SubChoice,RichFrac
0,1210,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, ...","[2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1211,"[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, ...","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1212,"[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1213,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, ...","[1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,1214,"[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, ...","[2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,1215,"[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, ...","[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,1216,"[0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, ...","[1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,1217,"[0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, ...","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,1218,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, ...","[1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
9,1219,"[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [39]:
def trial_label(x):
    as_strings = [num.astype(str) for num in x]
    return "".join(as_strings)

def make_features(trials_data):
    labeled_data = np.apply_along_axis(trial_label, 0, trials_data)
    combos = np.unique(labeled_data)
    string_to_index = dict(zip(combos, np.arange(len(combos))))
    map_to_index = np.vectorize(lambda x: string_to_index[x])
    mapped_data = map_to_index(labeled_data)
    return mapped_data

def get_marginal(x):
    """
    Helper function to compute and return marginal probability distribution for a 1d vector (x)
    """
    px = np.array([np.sum(x==xi) 
                      for xi in np.unique(x)])/len(x)
    return px

def get_joint(x, y):
    """
    Computes joint probability distribution between 1d vectors x and y
    """
    #  set up dictionary for joint distribution (x-->y-->freq)
    joint_x_y = {}
    
    for x_un in np.unique(x):
        joint_x_y[x_un] = dict(zip(np.unique(y), np.zeros(len(np.unique(y)))))
        
#    populate dictionary 
    for trial, x_val in enumerate(x):
        y_val = y[trial]
        joint_x_y[x_val][y_val] += 1
        
#   normalize to make distirbution  
    joint_sum = sum(sum(list(c.values())) for c in list(joint_x_y.values()))
    
    for key1 in joint_x_y:
        for key2 in joint_x_y[key1]:
            joint_x_y[key1][key2] /= joint_sum
            
    return joint_x_y

def mutual_inf(x, y):
    """
    Calculates the mutual information I(x;y)
    Assuming x,y are both [n x 1] dimensional
    """  
#     Calculate marginal distributions
    px = get_marginal(x)
    py = get_marginal(y)
    
    
    joint_x_y = get_joint(x,y)
# calculate mutual information
    mi = 0
    
    for n_x, x_un in enumerate(np.unique(x)):
        pxi = px[n_x] # p(x)
        
        for n_y, y_un in enumerate(np.unique(y)):
            pyi = py[n_y] # p(y)            
            
            joint_i = joint_x_y[x_un][y_un] # P(x,y)
            
            if ((pxi == 0) or (pyi == 0) or (joint_i ==0 )):
                continue
            else:
                mi += joint_i * np.log2(joint_i/(pxi*pyi))
                
    return mi

def deltaBound(ib_ipast,ib_ifuture,p_ipast,p_ifuture):
    ''' Function to calculate distance from the bound between an empirical IB and participant predictive info
    ib_ipast: ipast of empirical IB (x of convex hull)
    ib_ifuture: ifuture of empirical IB (y of convex hull)
    p_ipast: participant ipast (uncorrected)
    p_ifuture: participant ifuture (uncorrected)
    
    Returns participant ifuture minus the empirical bound (more negative = farther away from the bound)
    '''
    ind = np.argwhere(np.array(ib_ipast) > p_ipast)[0][0]
    slp = (ib_ifuture[ind]-ib_ifuture[ind-1])/(ib_ipast[ind]-ib_ipast[ind-1])
    intercept = ib_ifuture[ind]-(slp*ib_ipast[ind])
    #Return distance between participant Ifuture and interpolated bound - higher 
    return p_ifuture - ((p_ipast*slp)+intercept)

[0.015541405478960683, 0.05279415763506657]

In [123]:
def mi_per_subject(subject_data):
    trial_data = np.array([subject_data['Reward'].values, subject_data['SubChoice'].values])
    trial_data = np.array([[(subject_data['Reward'].values)[0]], 
                           [(subject_data['SubChoice'].values)[0]], 
                           [(subject_data['RichFrac'].values)[0]]])

    trial_data = trial_data.squeeze()
    features = make_features(trial_data)
    resp = trial_data[1,:]
    predInfo = [mutual_inf(features[:-2],resp[2:]),mutual_inf(features[1:],resp[:-1])]
    return predInfo

In [149]:
sub_dict = {}
for subject in np.unique(agg_df['Subject']):
    sub_dict[subject] = {}
    sub_dict[subject]['Subject'] = subject
    mi = mi_per_subject(agg_df[agg_df['Subject']==subject])
    
    sub_dict[subject]['Ipast'] = mi[0]
    sub_dict[subject]['Ifuture'] = mi[1]

In [150]:
sub_to_mi_df = pd.DataFrame.from_dict(sub_dict, orient='index')

In [159]:
merged_df= sub_to_mi_df.join(depressed ,lsuffix='_caller', rsuffix='_other')
merged_df

,Subject,Ifuture,Ipast,Group
1210,1210,0.052794,0.015541,MDD
1211,1211,0.261090,0.320899,MDD
1212,1212,0.046290,0.031976,MDD
1213,1213,0.012089,0.020877,MDD
1214,1214,0.027169,0.046480,MDD
1215,1215,0.135307,0.169878,MDD
1216,1216,0.066240,0.045035,MDD
1217,1217,0.407147,0.456303,MDD
1218,1218,0.028009,0.023019,MDD
1219,1219,0.261312,0.230790,MDD


In [160]:
merged_df.groupby(['Group']).mean()

,Subject,Ifuture,Ipast
Group,,,
Healthy Controls,1432.5,0.166082,0.170776
MDD,1241.5,0.121341,0.119929


In [153]:
agg_df

,Subject,Reward,SubChoice,RichFrac
0,1210,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, ...","[2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1211,"[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, ...","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1212,"[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1213,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, ...","[1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,1214,"[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, ...","[2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,1215,"[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, ...","[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,1216,"[0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, ...","[1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,1217,"[0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, ...","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,1218,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, ...","[1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
9,1219,"[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [99]:
resp

array([2, 1, 2, ..., 2, 1, 1])

In [102]:
subject = read_data[0]
subject

{'R1': array([2, 1, 2, ..., 2, 1, 1], dtype=uint8),
 'R2': array([1, 2, 1, ..., 2, 2, 2], dtype=uint8),
 'Rw': array([1, 1, 0, ..., 1, 1, 0], dtype=uint8),
 'S2': array([1, 2, 1, ..., 1, 1, 1], dtype=uint8),
 'best_R1': array([0, 0, 0, ..., 1, 1, 1])}